In [1]:
# %env CUDA_VISIBLE_DEVICES=1

In [2]:
import accelerate

accelerator = accelerate.Accelerator(
    log_with="tensorboard", 
    project_dir=".xvla",
    fsdp_plugin=accelerate.FullyShardedDataParallelPlugin(),
)

In [3]:
from xvla_wlr.model import XVLA, XVLAProcessor, Trainer, get_peft_model, Action, Observation

model = XVLA.from_pretrained("2toINF/X-VLA-SoftFold")
processor = XVLAProcessor.from_pretrained("2toINF/X-VLA-SoftFold", use_fast=True)

Florence2ForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [4]:
model = get_peft_model(model)
model, processor = accelerator.prepare(model, processor)
trainer = Trainer(model, processor, accelerator=accelerator)

In [5]:
import os

from datasets_wlr import WLRZhuangEpisodeDataset
from curobo.types.robot import RobotConfig
from xvla_wlr.model import DATA_DOMAIN_ID


dataset = WLRZhuangEpisodeDataset("samples/2026-01-21_demo_clothes/episode_0/data.json")
# TODO rm
# robot_config = RobotConfig.from_basic(
#     f"{os.getcwd()}/robots/piper_ros/src/piper_description/urdf/piper_description.urdf",
#     base_link="base_link",
#     ee_link="link8",
# )
domain_id = DATA_DOMAIN_ID["robomind-agilex"]


from xvla_wlr.dataset import XVLAWLRZhuangEpisodeDataset

# TODO
xvla_dataset = XVLAWLRZhuangEpisodeDataset(
    dataset=dataset,
    robot_config_left=RobotConfig.from_basic(
        f"{os.getcwd()}/robots/piper-dualarm/piper-dualarm.urdf",
        base_link="common_base_link",
        ee_link="left_link8",
    ),
    robot_config_right=RobotConfig.from_basic(
        f"{os.getcwd()}/robots/piper-dualarm/piper-dualarm.urdf",
        base_link="common_base_link",
        ee_link="right_link8",
    ),
    domain_id=domain_id,
    prefetch=True,
)

kinematics_fused_cu not found, JIT compiling...
geom_cu binary not found, jit compiling...
lbfgs_step_cu not found, JIT compiling...
line_search_cu not found, JIT compiling...
tensor_step_cu not found, jit compiling...


In [6]:
from xvla_wlr.dataset import Action


dataset = xvla_dataset[:32]

while True:
    num_timesteps_per_iteration = 16
    num_timesteps_remaining = len(dataset)

    while True:
        observation = dataset[:num_timesteps_remaining]
        if len(observation) < num_timesteps_per_iteration:
            break

        action = Action.from_observation(
            observation,
            num_steps=num_timesteps_per_iteration,
        )

        action_next = action[1:]
        observation_current = observation[:len(action_next)]

        num_timesteps_remaining -= len(action)

        trainer.fit(
            observation=observation_current,
            action=action_next,
        )

OutOfMemoryError: CUDA out of memory. Tried to allocate 324.00 MiB. GPU 0 has a total capacity of 22.02 GiB of which 150.00 MiB is free. Process 3603106 has 500.00 MiB memory in use. Including non-PyTorch memory, this process has 20.71 GiB memory in use. Of the allocated memory 19.53 GiB is allocated by PyTorch, and 748.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)